This program read the preprocessed data, in .npy format

use Convnet to train a model

This jupyter notebook is built with these previous contributors:
1. https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d Keras Model for Beginners (0.210 on LB)+EDA+R&D by DeveshMaheshwari
2. https://www.kaggle.com/toregil/welcome-to-deep-learning-cnn-99 Welcome to deep learning (CNN 99%) by Peter Grenholm
3. https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/ Display Deep Learning Model Training History in Keras by Jason Brownlee
4. https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/ Evaluate the Performance Of Deep Learning Models in Keras by Jason Brownlee
5. https://machinelearningmastery.com/check-point-deep-learning-models-keras/ Add callbacks for best evaluation

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import median_filter
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
X_train_rotate = np.load("/home/ubuntu/X_train_rotate.npy")
target_train = np.load("/home/ubuntu/target_train.npy")

In [3]:
print(X_train_rotate.shape)
print(target_train.shape)

(4812, 75, 75, 2)
(4812,)


In [4]:
import keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 2)))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    gmodel.add(BatchNormalization())
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]


#file_path = ".model_weights.hdf5"
#callbacks = get_callbacks(filepath=file_path, patience=5)

#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [6]:
import timeit
start = timeit.default_timer()

In [7]:
#https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

# fix random seed for reproducibility

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

seed = 7
np.random.seed(seed)

gmodel = KerasClassifier(build_fn=getModel,
                         epochs = 10,
                         batch_size = 24,
                         verbose = 1,
                         )

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, random_state=seed)
results = cross_val_score(gmodel, X_train_rotate, target_train, cv=kfold)
print (results);


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        36928     
__________

482/482 [==============================] - 1s 1ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 73, 73, 64)        1216      
_________________________________________________________________
batch_normalization_9 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)   

4331/4331 [==============================] - 16s 4ms/step - loss: 0.2494 - acc: 0.9019
Epoch 10/10
481/481 [==============================] - 1s 1ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 73, 73, 64)        1216      
_________________________________________________________________
batch_normalization_17 (Batc (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_18 (Batc (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 35, 35, 64)       

4331/4331 [==============================] - 16s 4ms/step - loss: 0.2208 - acc: 0.9039
Epoch 9/10
4331/4331 [==============================] - 16s 4ms/step - loss: 0.2193 - acc: 0.9116
Epoch 10/10
481/481 [==============================] - 1s 2ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 73, 73, 64)        1216      
_________________________________________________________________
batch_normalization_25 (Batc (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_26 (Batc (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 35, 35, 64)        0         
______________________

4331/4331 [==============================] - 16s 4ms/step - loss: 0.3000 - acc: 0.8843
Epoch 8/10
4331/4331 [==============================] - 16s 4ms/step - loss: 0.2097 - acc: 0.9160
Epoch 9/10
4331/4331 [==============================] - 16s 4ms/step - loss: 0.2405 - acc: 0.9081
Epoch 10/10
481/481 [==============================] - 1s 2ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 73, 73, 64)        1216      
_________________________________________________________________
batch_normalization_33 (Batc (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_34 (Batc (None, 71, 71, 64)        256       
________________________________________________________

4332/4332 [==============================] - 16s 4ms/step - loss: 0.2867 - acc: 0.8869
Epoch 7/10
4332/4332 [==============================] - 16s 4ms/step - loss: 0.2307 - acc: 0.9058
Epoch 8/10
4332/4332 [==============================] - 16s 4ms/step - loss: 0.2046 - acc: 0.9137
Epoch 9/10
4332/4332 [==============================] - 16s 4ms/step - loss: 0.1975 - acc: 0.9217
Epoch 10/10
480/480 [==============================] - 1s 2ms/step
[ 0.81950208  0.8692946   0.69502075  0.75675676  0.79209979  0.73596673
  0.67983368  0.79002079  0.83575883  0.775     ]


[ 0.81950208  0.8692946   0.69502075  0.75675676  0.79209979  0.73596673
  0.67983368  0.79002079  0.83575883  0.775     ]


In [9]:
'''from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []

import os
gmodel=getModel()
for train_index, test_index in kfold.split(X_train_rotate, target_train):
    gmodel.fit(X_train_rotate[train_index], target_train[train_index], 
               batch_size = 24,
               epochs = 10,
               verbose = 1,
               validation_data=(X_train_rotate[test_index], target_train[test_index])
               callbacks=callbacks_list)
    
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) 
'''

'from sklearn.model_selection import StratifiedKFold\nkfold = StratifiedKFold(n_splits=5, shuffle=True)\ncvscores = []\n\nimport os\ngmodel=getModel()\nfor train_index, test_index in kfold.split(X_train_rotate, target_train):\n    gmodel.fit(X_train_rotate[train_index], target_train[train_index], \n               batch_size = 24,\n               epochs = 10,\n               verbose = 1,\n               callbacks=callbacks_list)\n    \n    scores = gmodel.evaluate(X_train_rotate[test_index], target_train[test_index])\n    print (scores[0])\n    cvscores.append(scores[0]*100)\n\nprint("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) \n'

In [10]:
'''X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train_rotate, target_train, random_state=1, train_size=0.9)
#Without denoising, core features.

import os
gmodel=getModel()
history = gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=20,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks_list)'''

'X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train_rotate, target_train, random_state=1, train_size=0.9)\n#Without denoising, core features.\n\nimport os\ngmodel=getModel()\nhistory = gmodel.fit(X_train_cv, y_train_cv,\n          batch_size=24,\n          epochs=20,\n          verbose=1,\n          validation_data=(X_valid, y_valid),\n          callbacks=callbacks_list)'

In [11]:
stop = timeit.default_timer()
print (stop-start)

1599.3307381349987


In [12]:
#gmodel.load_weights(filepath=filepath)
score = gmodel.evaluate(X_train_rotate, target_train, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

AttributeError: 'KerasClassifier' object has no attribute 'evaluate'

In [ ]:
'''print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()'''

In [ ]:
#import testing data
X_test = np.load("/home/ubuntu/X_test.npy")
test_id = np.load('/home/ubuntu/test_id.npy')
predicted_test=gmodel.predict_proba(X_test)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)